# Business Problem

### Objective

The objective of this capstone project is to analyse and select the best locations in the city of Calgary, Canada to open a new restaurant. This will be a preliminary overview of the data providing the framework for a more detailed analysis later. 

### Target Audience

The target audience of this project is entrepreneurs looking to get an overview of the current restaurant scene in Calgary with the eventual goal of obtaining financing and/or investors to open a new establishment.

### Data

Data will be obtained from the City of Calgary 2019 census. The original data will be cleaned and sorted using SQL queries so only the relevant data is imported.
The census data that is of interest is the type (ie: residential), community, community population, and community location.
The census data is already grouped by community with a total of 212 unique values. The original location data was stored as a multipolygon that was split into two columns for the latitude and longitude location information. The multipolygon could be used with the population count to calculate a relative population density later. Although the census data contains a lot of useful data for population density and demographic distributions, it is outside the scope of this initial investigation.
After the census data has been wrangled, Foursquare API will be used to get the venue data in the respective neighborhoods. The venue data will be combined with the census data and then passed through the Scikit-Learn k-means clustering algorithm to reveal trends and patterns in the data distribution. The k-means clustering algorithm will combine the neighborhoods into groups that share similar attributes based on the venue data returned from Foursquare. From there, the individual clusters will be analysed to determine to most suitable location for a new restaurant based on current restaurant locations and distribution density.



### Methodology

#### Import Libraries

In [1]:
import pandas as pd
from pandas.io.json import json_normalize

import requests

import numpy as np

import geocoder
from geopy.geocoders import Nominatim

from sklearn.cluster import KMeans
from sklearn.preprocessing import StandardScaler

import matplotlib.cm as cm
import matplotlib.colors as colors

import folium

#### Import census data

In [2]:
# 2019 Census data after running SQL query to obtain relevant data
calgary = pd.read_csv('Census_2019_Query.csv')
calgary.head()

,CLASS,CLASS_CODE,COMM_CODE,NEIGHBORHOOD,RES_CNT,LATITUDE,LONGITUDE
0,Residential,1,ABB,ABBEYDALE,5957,51.066971,-113.935119
1,Residential,1,ACA,ACADIA,10520,50.980779,-114.050012
2,Residential,1,ALB,ALBERT PARK/RADISSON HEIGHTS,6997,51.045153,-113.981603
3,Residential,1,ALT,ALTADORE,6942,51.023194,-114.100546
4,Residential,1,APP,APPLEWOOD PARK,6981,51.045203,-113.921220


#### Use geopy library to get lat/long of Calgary

In [3]:
address = 'Calgary, AB'

geolocator = Nominatim(user_agent="to_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinates of Calgary are {}, {}.'.format(latitude, longitude))

The geograpical coordinates of Calgary are 51.0534234, -114.0625892.


#### Create a map of Calgary with neighborhoods superimposed on top

In [4]:
# create map of Calgary using latitude and longitude values
map_calgary = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighborhood in zip(calgary['LATITUDE'], calgary['LONGITUDE'], calgary['NEIGHBORHOOD']):
    label = '{}'.format(neighborhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_calgary)  
    
map_calgary

#### Load FourSquare account credentials

In [5]:
CLIENT_ID = 'xxx'
CLIENT_SECRET = 'xxx'
VERSION = '20180605'
categoryID = '4d4b7105d754a06374d81259' # Foursquare food category

### Explore Calgary Neighborhoods

In [6]:
def getNearbyVenues(names, latitudes, longitudes, radius=500, limit=100):
    
    venues_list=[]
    for name, lat, lng in zip(names, latitudes, longitudes):
        print(name)
            
        # create the API request URL
        url = 'https://api.foursquare.com/v2/venues/explore?&client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}'.format(
            CLIENT_ID, 
            CLIENT_SECRET, 
            VERSION, 
            lat, 
            lng, 
            radius, 
            limit)
            
        # make the GET request
        results = requests.get(url).json()["response"]['groups'][0]['items']
        
        # return only relevant information for each nearby venue
        venues_list.append([(
            name, 
            lat, 
            lng, 
            v['venue']['name'], 
            v['venue']['location']['lat'], 
            v['venue']['location']['lng'],  
            v['venue']['categories'][0]['name']) for v in results])

    nearby_venues = pd.DataFrame([item for venue_list in venues_list for item in venue_list])
    nearby_venues.columns = ['Neighborhood', 
                  'Neighborhood Latitude', 
                  'Neighborhood Longitude', 
                  'Venue', 
                  'Venue Latitude', 
                  'Venue Longitude', 
                  'Venue Category']
    
    return(nearby_venues)

In [7]:
calgary_venues = getNearbyVenues(names = calgary['NEIGHBORHOOD'],
                                   latitudes = calgary['LATITUDE'],
                                   longitudes = calgary['LONGITUDE'])
calgary_venues.head()

ABBEYDALE
ACADIA
ALBERT PARK/RADISSON HEIGHTS
ALTADORE
APPLEWOOD PARK
ARBOUR LAKE
ASPEN WOODS
AUBURN BAY
BANFF TRAIL
BANKVIEW
BAYVIEW
BEDDINGTON HEIGHTS
BEL-AIRE
BELMONT
BELTLINE
BELVEDERE
BONAVISTA DOWNS
BOWNESS
BRAESIDE
BRENTWOOD
BRIDGELAND/RIVERSIDE
BRIDLEWOOD
BRITANNIA
CAMBRIAN HEIGHTS
CANYON MEADOWS
CAPITOL HILL
CARRINGTON
CASTLERIDGE
CEDARBRAE
CHAPARRAL
CHARLESWOOD
CHINATOWN
CHINOOK PARK
CHRISTIE PARK
CITADEL
CITYSCAPE
CLIFF BUNGALOW
COACH HILL
COLLINGWOOD
COPPERFIELD
CORAL SPRINGS
CORNERSTONE
COUGAR RIDGE
COUNTRY HILLS
COUNTRY HILLS VILLAGE
COVENTRY HILLS
CRANSTON
CRESCENT HEIGHTS
CRESTMONT
CURRIE BARRACKS
DALHOUSIE
DEER RIDGE
DEER RUN
DIAMOND COVE
DISCOVERY RIDGE
DOUGLASDALE/GLEN
DOVER
DOWNTOWN COMMERCIAL CORE
DOWNTOWN EAST VILLAGE
DOWNTOWN WEST END
EAGLE RIDGE
EAU CLAIRE
EDGEMONT
ELBOW PARK
ELBOYA
ERIN WOODS
ERLTON
EVANSTON
EVERGREEN
FAIRVIEW
FALCONRIDGE
FOREST HEIGHTS
FOREST LAWN
GARRISON GREEN
GARRISON WOODS
GLAMORGAN
GLENBROOK
GLENDALE
GREENVIEW
GREENWOOD/GREENBRIAR
HAMPTON

,Neighborhood,Neighborhood Latitude,Neighborhood Longitude,Venue,Venue Latitude,Venue Longitude,Venue Category
0,ABBEYDALE,51.066971,-113.935119,A&W,51.068291,-113.933571,Fast Food Restaurant
1,ABBEYDALE,51.066971,-113.935119,Calgary Co-op,51.068719,-113.934014,Grocery Store
2,ABBEYDALE,51.066971,-113.935119,Subway,51.069623,-113.932907,Sandwich Place
3,ABBEYDALE,51.066971,-113.935119,Subway,51.069589,-113.933284,Sandwich Place
4,ABBEYDALE,51.066971,-113.935119,Petro-Canada,51.069370,-113.933826,Gas Station


#### Analyze Each Neighborhood

In [8]:
# one hot encoding
calgary_onehot = pd.get_dummies(calgary_venues[['Venue Category']], prefix="", prefix_sep="")

# add neighborhood column back to dataframe
calgary_onehot['Neighborhood'] = calgary_venues['Neighborhood'] 

# move neighborhood column to the first column
fixed_columns = [calgary_onehot.columns[-1]] + list(calgary_onehot.columns[:-1])
calgary_onehot = calgary_onehot[fixed_columns]

calgary_onehot.head()

,Zoo Exhibit,American Restaurant,Argentinian Restaurant,Arts & Crafts Store,Asian Restaurant,Athletics & Sports,Auto Garage,BBQ Joint,Bakery,Bank,...,Vegetarian / Vegan Restaurant,Video Game Store,Video Store,Vietnamese Restaurant,Warehouse Store,Water Park,Wine Bar,Wine Shop,Women's Store,Yoga Studio
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [9]:
calgary_grouped = calgary_onehot.groupby('Neighborhood').mean().reset_index()

#### Create new DataFrame for restaurants only

In [10]:
calgary_restaurant = calgary_grouped[['Neighborhood', 'Restaurant']]
calgary_restaurant

,Neighborhood,Restaurant
0,ABBEYDALE,0.000000
1,ACADIA,0.066667
2,ALBERT PARK/RADISSON HEIGHTS,0.000000
3,ALTADORE,0.000000
4,APPLEWOOD PARK,0.000000
...,...,...
184,WINSTON HEIGHTS/MOUNTVIEW,0.000000
185,WOLF WILLOW,0.000000
186,WOODBINE,0.000000
187,WOODLANDS,0.000000


### Cluster Neighborhoods

#### Run k-means to cluster the neighborhood into 6 clusters

In [11]:
# set number of clusters
kclusters = 6

calgary_grouped_clustering = calgary_restaurant.drop('Neighborhood', 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(calgary_grouped_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10] 

array([0, 3, 0, 0, 0, 0, 0, 0, 0, 0])

#### Create a new dataframe that includes the clusters as well as restaurant frequency for each neighborhood

In [13]:
# add clustering labels
calgary_restaurant.insert(0, 'Cluster Labels', kmeans.labels_)

calgary_merged = calgary

# merge calgary_grouped with calgary to add latitude/longitude for each neighborhood
calgary_merged = calgary_merged.join(calgary_restaurant.set_index('Neighborhood'), on='NEIGHBORHOOD')

# cast Cluster Labels Column to int from float
calgary_merged['Cluster Labels'] = calgary_merged['Cluster Labels'].fillna(0).astype(np.int64)

# save to CSV for reference
calgary_merged.to_csv('calgary_merged.csv', index = False)

calgary_merged.head()

,CLASS,CLASS_CODE,COMM_CODE,NEIGHBORHOOD,RES_CNT,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
0,Residential,1,ABB,ABBEYDALE,5957,51.066971,-113.935119,0,0.000000
1,Residential,1,ACA,ACADIA,10520,50.980779,-114.050012,3,0.066667
2,Residential,1,ALB,ALBERT PARK/RADISSON HEIGHTS,6997,51.045153,-113.981603,0,0.000000
3,Residential,1,ALT,ALTADORE,6942,51.023194,-114.100546,0,0.000000
4,Residential,1,APP,APPLEWOOD PARK,6981,51.045203,-113.921220,0,0.000000


#### Visualize the resulting clusters

In [14]:
# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i + x + (i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(calgary_merged['LATITUDE'], calgary_merged['LONGITUDE'], calgary_merged['NEIGHBORHOOD'], calgary_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
 
    
map_clusters

In [15]:
# sort the results by Restaurant freq
print(calgary_merged.shape)
calgary_merged.sort_values(['Restaurant'], inplace=True)
calgary_merged

(212, 9)


,CLASS,CLASS_CODE,COMM_CODE,NEIGHBORHOOD,RES_CNT,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
0,Residential,1,ABB,ABBEYDALE,5957,51.066971,-113.935119,0,0.0
129,Residential,1,OAK,OAKRIDGE,5690,50.967399,-114.117930,0,0.0
130,Residential,1,OGD,OGDEN,8576,50.999734,-114.013082,0,0.0
131,Residential,1,PAL,PALLISER,3672,50.963194,-114.106491,0,0.0
132,Residential,1,PAN,PANORAMA HILLS,25710,51.166095,-114.068814,0,0.0
...,...,...,...,...,...,...,...,...,...
163,Residential,1,SAN,SANDSTONE VALLEY,5904,51.142510,-114.090962,0,NaN
167,Residential,1,SET,SETON,1134,50.870782,-113.929450,0,NaN
180,Residential,1,SPH,SPRINGBANK HILL,9943,51.037846,-114.222561,0,NaN
187,Residential,1,TAR,TARADALE,19026,51.125257,-113.921268,0,NaN


### Examine Clusters
#### Cluster 0

In [16]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 0, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
0,1,51.066971,-113.935119,0,0.0
129,1,50.967399,-114.117930,0,0.0
130,1,50.999734,-114.013082,0,0.0
131,1,50.963194,-114.106491,0,0.0
132,1,51.166095,-114.068814,0,0.0
...,...,...,...,...,...
163,1,51.142510,-114.090962,0,NaN
167,1,50.870782,-113.929450,0,NaN
180,1,51.037846,-114.222561,0,NaN
187,1,51.125257,-113.921268,0,NaN


#### Cluster 1

In [17]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 1, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
196,1,51.102261,-114.259479,1,0.5
94,1,50.994095,-114.092686,1,0.5
60,1,50.989625,-114.094888,1,0.5
171,1,51.168038,-114.161064,1,0.5


#### Cluster 2

In [18]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 2, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
64,1,51.017444,-114.071523,2,0.20
14,1,51.040340,-114.045847,2,0.20
73,1,51.004556,-114.125794,2,0.20
206,1,51.008716,-114.081518,2,0.25


#### Cluster 3

In [19]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 3, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
47,1,51.066977,-114.063250,3,0.032258
27,1,51.100342,-113.958257,3,0.035714
114,1,50.931932,-113.975487,3,0.037037
151,1,51.028854,-114.071527,3,0.041667
74,1,51.023194,-114.112496,3,0.043478
157,1,51.029099,-114.068988,3,0.043478
120,1,51.036058,-114.061246,3,0.050000
57,1,51.050419,-114.083170,3,0.051724
66,1,51.035866,-114.063036,3,0.052632
197,1,51.085431,-114.128995,3,0.052632


#### Cluster 4

In [20]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 4, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
134,1,51.023774,-114.066824,4,0.333333


#### Cluster 5

In [21]:
calgary_merged.loc[calgary_merged['Cluster Labels'] == 5, calgary_merged.columns[[1] + list(range(5, calgary_merged.shape[1]))]]

,CLASS_CODE,LATITUDE,LONGITUDE,Cluster Labels,Restaurant
63,1,51.030041,-114.074988,5,0.100000
109,1,51.052515,-113.978076,5,0.142857
186,1,51.058743,-114.066984,5,0.142857
24,1,50.939443,-114.069348,5,0.166667


#### Results

The neighborhoods were split into 6 clusters using the Scikit-Learn k-means clustering algorithm based on restaurant frequency. Cluster 1 contained the highest frequency with 0.5 and a total of 4 neighborhoods. Most of the neighborhoods fell into cluster 0 with a frequency of 0 restaurants for 181 of the total 213 neighborhoods (85% of the neighborhoods).

#### Discussion

While most of the neighborhoods do not contain any restaurants, this does not make them a suitable candidate for a new restaurant. Further analysis needs to be done on the type and distribution of restaurants based on what type of restaurant the stakeholders are looking to invest in. Further analysis may include calculating the population density from the multipolygon information and superimposing that on the neighborhood map along with the distribution of the type of restaurant being investigated. Another avenue of investigation could be population demographic information and choosing a suitable restaurant type based on a combination of census and polling data.

#### Conclusion

The 2019 City of Calgary census data, combined with Foursquare API data, provided enough information for a preliminary analysis of restaurant distribution. Most of the neighborhoods in Calgary do not contain any restaurants. This, however does not make them suitable candidates for a new restaurant. Further investigation needs to be completed to provide a complete picture of all the contributing factors.
